In [2]:
from string import digits
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import regex as re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [16]:
df1 = pd.read_pickle('./indeed_data_related_jobs')
df1.dropna(subset=['company_name', 'summary'], inplace=True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3016 entries, 0 to 3699
Data columns (total 6 columns):
job_category    3016 non-null object
job_title       3016 non-null object
company_name    3016 non-null object
location        3016 non-null object
summary         3016 non-null object
salary          124 non-null object
dtypes: object(6)
memory usage: 164.9+ KB


In [17]:
# convert all to small letters if string
df1 = df1.applymap(lambda x: x.lower().strip() if isinstance(x, str) else x)
# remove all non-alphabets
df1.job_title = df1.job_title.map(lambda x: re.sub(r'[^A-Za-z\s]','',x).strip())
df1.location = df1.location.map(lambda x: re.sub(r'[^A-Za-z\s]','',x).strip())
df1.summary = df1.summary.map(lambda x: re.sub(r'[^A-Za-z\s]','',x).strip())
# remove business licence numbers
df1.company_name = df1.company_name.map(lambda x: x[:x.index(', ea licence')] if x.find(', ea licence') != -1 else x)

In [49]:
# Extract those with salary data
df1_salaried = df1[~df1.salary.isnull()]
df1_salaried.reset_index(drop=True, inplace=True)

In [50]:
# Salary only comes in monthly and yearly format
df1_salaried.salary.map(lambda x: x[-8:]).value_counts()

 a month    118
0 a year      6
Name: salary, dtype: int64

In [54]:
df1_salaried.salary

0         $5,000 - $8,500 a month
1        $36,000 - $72,000 a year
2         $2,500 - $4,000 a month
3        $5,000 - $14,000 a month
4                  $4,300 a month
5         $3,000 - $5,000 a month
6         $3,600 - $4,000 a month
7         $4,000 - $7,000 a month
8         $1,300 - $1,600 a month
9                  $3,500 a month
10        $4,000 - $6,000 a month
11        $3,800 - $4,200 a month
12        $4,500 - $5,500 a month
13        $5,000 - $6,500 a month
14        $4,700 - $5,000 a month
15        $2,400 - $3,000 a month
16                 $4,500 a month
17        $2,500 - $4,200 a month
18        $2,200 - $3,000 a month
19        $1,800 - $2,600 a month
20        $4,000 - $6,000 a month
21        $5,000 - $8,000 a month
22        $4,800 - $7,000 a month
23     $100,000 - $125,000 a year
24       $21,000 - $50,000 a year
25        $2,000 - $3,000 a month
26       $4,600 - $12,600 a month
27        $3,000 - $6,000 a month
28        $2,500 - $4,000 a month
29        $3,5

In [64]:
# convert all salary into yearly format
salary_range = df1_salaried.salary.map(lambda x: re.sub('[^0-9\s]', '', ' '.join(re.findall(r'\d+(?:[\d,.]*\d)', x))))
salary_period = df1_salaried.salary.map(lambda x: x[x.find('month'):] if x.find('month') >= 0 else 'year')

In [70]:
salary_range.map(lambda x: x.split())

AttributeError: 'list' object has no attribute 'mean'